### Importing the Necessary Packages

In [1]:
from typing_extensions import Literal, Annotated 
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, StateGraph,START,END
from langgraph.types import Command
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
import os

In [2]:
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
LANGCHAIN_TRACING = os.getenv("LANGCHAIN_TRACING")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

os.environ["LANGCHAIN_TRACING"] = LANGCHAIN_TRACING
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "llama-3.3-70b-versatile")

In [5]:
llm.invoke("Hello do you think I am a good person?")

AIMessage(content='That\'s a thoughtful question. I\'m happy to chat with you, but I want to clarify that I\'m a large language model, I don\'t have personal opinions or judgments about individuals. My purpose is to provide information, answer questions, and engage in conversations to the best of my abilities.\n\nThat being said, I believe that every person has the potential to be good and make positive contributions to the world. Being a "good person" is often subjective and can depend on various factors, such as personal values, actions, and intentions.\n\nIf you\'re wondering about your own character, I\'d encourage you to reflect on your values, behaviors, and relationships. Ask yourself:\n\n* What are my core values, and do I live by them?\n* How do I treat others, and do I strive to be kind and compassionate?\n* What are my strengths and weaknesses, and how can I work on improving myself?\n\nRemember that personal growth and self-improvement are lifelong journeys. It\'s okay to m

In [6]:
llm2 = ChatGroq(model_name = "deepseek-r1-distill-llama-70b")

In [7]:
llm2.invoke("Hello do you think I am a good person?")

AIMessage(content='<think>\nOkay, so I\'m trying to figure out if I\'m a good person. Hmm, where do I even start? I guess I should think about what makes someone "good." Maybe it\'s about how they treat others, or maybe it\'s about their actions and choices. I\'m not really sure. Let me break this down.\n\nFirst, I think about my interactions with others. Do I treat people kindly? Well, sometimes I\'m nice, but there are times when I\'m stressed or upset and maybe I\'m not as patient as I could be. Like, I remember a time when my friend was talking about something they were excited about, and I kind of tuned out because I was preoccupied with my own stuff. That wasn\'t very considerate. But then again, I also remember helping someone move last weekend, which was a bit of a hassle, but I did it anyway. So, it\'s a mix, I guess.\n\nThen there\'s the aspect of honesty. I try to be truthful, but I\'m not perfect. There was that one time I told a white lie to get out of a social event I did

We are going to create two agents, one of the agents is an addition expert and one of the agents is a multiplication agent. 
The only thing is that addition agent cannot perform multiplication and vice-versa. Therefore if we give it a complex equation to solve which consists of both addition and multiplication, it will delegate the tasks accordingly. 

This is a very simple example just to understand the workflow of the network system.

In [8]:
def add_numbers(state):
    result = state["num1"] + state["num2"]
    print(f"Addition Expert: The sum of {state['num1']} and {state['num2']} is {result}")
    return Command(goto="multiply", update = {"sum": result})

In [9]:
@tool 
def transfer_to_multiplication_expert():
    """ Ask the multiplication agent for help """
    return 

In [10]:
@tool 
def transfer_to_addition_expert():
    """ Ask the addition agent for help """
    return 

In [11]:
model_with_tool = llm.bind_tools([transfer_to_multiplication_expert])

In [12]:
ai_message = model_with_tool.invoke("how are you today?")

In [13]:
ai_message.tool_calls

[]

In [14]:
def additional_expert(state: MessagesState) -> Command[Literal["multiplication_expert", "__end__"]]:
    
    system_prompt = (
        "You are an expert addition agent. You can only add two numbers together. "
        "You will receive the numbers in the state. "
        "You must return the sum of the numbers in the state as 'sum'. "
        "After you have added the two numbers, you must transfer control to the multiplication expert."
        "Always do your portions of the task to the best of your ability before the handoff."
    ) 

    messages = [{
        "role": "system",
        "content": system_prompt
    }] + state["messages"]

    ai_msg = llm.bind_tools([transfer_to_multiplication_expert]).invoke(messages)

    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred to multiplication expert.",
            "tool_call_id": tool_call_id,
        }   
        return Command(goto="multiplication_expert", update={"messages": [ai_msg, tool_msg]})
    
    return {"messages": [ai_msg]}

In [15]:
def multiplication_expert(state: MessagesState) -> Command[Literal["additional_expert", "__end__"]]:
    
    system_prompt = (
        "You are an expert multiplication agent, you can ask the addition expert for help with addition"
        "After you have multiplied the two numbers, you must transfer control to the addition expert."
        "Always do your portions of the task to the best of your ability before the handoff."
    )

    messages = [{
        "role": "system",
        "content": system_prompt
    }] + state["messages"]

    ai_msg = llm2.bind_tools([transfer_to_addition_expert]).invoke(messages)

    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred to addition expert.",
            "tool_call_id": tool_call_id,
        }   
        return Command(goto="additional_expert", update={"messages": [ai_msg, tool_msg]})
    return {"messages": [ai_msg]}

In [16]:
graph = StateGraph(MessagesState)

In [17]:
graph.add_node("additional_expert",additional_expert)

In [18]:
graph.add_node("multiplication_expert",multiplication_expert)

In [19]:
graph.add_edge(START,"additional_expert")

In [20]:
app = graph.compile()

In [21]:
# display(Image(app.get_graph().draw_mermaid_png()))

In [22]:
app.invoke({"messages":[("user","what's (3 + 5) * 12. Provide me the output")]})

{'messages': [HumanMessage(content="what's (3 + 5) * 12. Provide me the output", additional_kwargs={}, response_metadata={}, id='4ba4504e-a432-41ee-a957-c54a36f9f435'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'r0s9kfpgz', 'function': {'arguments': '{"sum":8}', 'name': 'transfer_to_multiplication_expert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 301, 'total_tokens': 319, 'completion_time': 0.057563357, 'prompt_time': 0.025417995, 'queue_time': 0.209828801, 'total_time': 0.082981352}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2626918a-b1d2-47c6-9596-529f8ac4513f-0', tool_calls=[{'name': 'transfer_to_multiplication_expert', 'args': {'sum': 8}, 'id': 'r0s9kfpgz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 301, 'output_tokens': 18, 'total_tokens': 319}),
  ToolMessage(con

In [23]:
from langchain_core.messages import convert_to_messages
def pretty_print_messages(update):
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")

    for node_name, node_update in update.items():
        print(f"Update from node {node_name}:")
        print("\n")

        for m in convert_to_messages(node_update["messages"]):
            m.pretty_print()
        print("\n")

In [24]:
# Let's run the graph with an expression that requires both addition and multiplication:
for chunk in app.stream(
    {"messages": [("user", "what's (3 + 5) * 12. Provide me the output")]},
):
    print("****chunk...****")
    
    pretty_print_messages(chunk)

****chunk...****
Update from node additional_expert:


================================== Ai Message ==================================
Tool Calls:
  transfer_to_multiplication_expert (d0qqv8kkw)
 Call ID: d0qqv8kkw
  Args:
    sum: 8
================================= Tool Message =================================

Successfully transferred to multiplication expert.


****chunk...****
Update from node multiplication_expert:


================================== Ai Message ==================================

The result of (3 + 5) * 12 is 96. 

Step-by-step explanation:
1. First, compute the addition inside the parentheses: 3 + 5 = 8.
2. Next, multiply the sum by 12: 8 * 12 = 96.

Answer: 96




In [25]:
@tool
def transfer_to_travel_advisor():
    """ Ask the travel advisor agent for help """
    return 

In [26]:
@tool
def transfer_to_hotel_advisor():
    """ Ask the hotel advisor agent for help """
    return

In [27]:
def travel_advisor(state: MessagesState) -> Command[Literal["hotel_advisor", "__end__"]]:

    system_prompt = (
        "You are a general travel expert that can recommend travel destinations like Paris, New York, Tokyo, etc. "
        "If you need hotel recommendations, you must ask 'hotel_advisor' for help"
    )

    messages = [{
        "role": "system",
        "content": system_prompt
    }] + state["messages"]

    ai_msg = llm.bind_tools([transfer_to_hotel_advisor]).invoke(messages)

    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred to hotel advisor.",
            "tool_call_id": tool_call_id,
        }   
        return Command(goto="hotel_advisor", update={"messages": [ai_msg, tool_msg]})
    return {"messages": [ai_msg]}

In [28]:
def hotel_advisor(state: MessagesState) -> Command[Literal["travel_advisor", "__end__"]]:

    system_prompt = (
        "You are a hotel expert that can recommend hotels in various cities. "
        "You can only recommend hotels, you cannot recommend travel destinations. "
        "If you need to recommend a travel destination, you must ask 'travel_advisor' for help"
    )

    messages = [{
        "role": "system",
        "content": system_prompt
    }] + state["messages"]

    ai_msg = llm2.bind_tools([transfer_to_travel_advisor]).invoke(messages)

    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred to travel advisor.",
            "tool_call_id": tool_call_id,
        }   
        return Command(goto="travel_advisor", update={"messages": [ai_msg, tool_msg]})
    return {"messages": [ai_msg]}

In [29]:
graph_builder = StateGraph(MessagesState)

In [30]:
graph_builder.add_node("travel_advisor",travel_advisor)
graph_builder.add_node("hotel_advisor",hotel_advisor)

In [31]:
graph_builder.add_edge(START,"travel_advisor")

In [32]:
new_app = graph_builder.compile()

In [ ]:
# display(Image(new_app.get_graph().draw_mermaid_png()))

In [33]:
new_app.invoke({"messages":[("user","I am planning a trip to the Paris from Mumbai. Can you guide me on travel options and suggest the best hotel?")]})

{'messages': [HumanMessage(content='I am planning a trip to the Paris from Mumbai. Can you guide me on travel options and suggest the best hotel?', additional_kwargs={}, response_metadata={}, id='e8b74fb2-922e-4dd3-8745-788e44aa964c'),
  AIMessage(content='To travel from Mumbai to Paris, you have several options. You can fly from Chhatrapati Shivaji Maharaj International Airport (BOM) in Mumbai to Paris Charles de Gaulle Airport (CDG) in Paris. Airlines such as Air France, Air India, and Emirates offer direct or connecting flights. Alternatively, you can also consider flying into Paris Orly Airport (ORY), which is another major airport serving the city.\n\nAs for the best hotel in Paris, I would recommend asking a hotel advisor for more specific and tailored advice, as they can provide the most up-to-date and detailed information on hotels that fit your budget, preferences, and the purpose of your trip.\n\n', additional_kwargs={'tool_calls': [{'id': 'rk7mm8kwc', 'function': {'arguments

In [38]:
for chunk in new_app.stream(
    {"messages": [("user", "I am planning a trip to the Paris from Mumbai. Can you guide me on travel options and suggest the best hotel?")]},
):
    print("****chunk****")
    
    pretty_print_messages(chunk)

****chunk****
Update from node travel_advisor:


================================== Ai Message ==================================

To travel from Mumbai to Paris, you have several options. You can fly from Chhatrapati Shivaji Maharaj International Airport (BOM) in Mumbai to Paris Charles de Gaulle Airport (CDG) or Paris Orly Airport (ORY) with airlines such as Air France, Emirates, or Lufthansa. The flight duration is approximately 10-11 hours, depending on the airline and route. Alternatively, you can also consider flying with a layover, which may be cheaper but takes longer.

For the best hotel recommendations in Paris, I would like to ask the hotel advisor for help.
Tool Calls:
  transfer_to_hotel_advisor (7g2x9fyxd)
 Call ID: 7g2x9fyxd
  Args:
================================= Tool Message =================================

Successfully transferred to hotel advisor.


****chunk****
Update from node hotel_advisor:


================================== Ai Message =====================

In [37]:
for chunk in new_app.stream(
    {"messages": [("user", "What are the best flight options from Mumbai to California, and can you recommend top hotels for a comfortable and convenient stay?")]},
):
    print("****chunk****")
    
    pretty_print_messages(chunk)

****chunk****
Update from node travel_advisor:


================================== Ai Message ==================================

To find the best flight options from Mumbai to California, I can suggest searching for flights on various airlines such as Air India, Emirates, or Lufthansa. However, for real-time information and to get the most up-to-date flight schedules and prices, I recommend checking with the airlines directly or using a flight booking website.

For hotel recommendations, I'd be happy to ask the hotel advisor agent for help.
Tool Calls:
  transfer_to_hotel_advisor (3em9r364f)
 Call ID: 3em9r364f
  Args:
================================= Tool Message =================================

Successfully transferred to hotel advisor.


****chunk****
Update from node hotel_advisor:


================================== Ai Message ==================================

Sure! For flight options from Mumbai to California, I'll need to transfer your request to the travel advisor. For 